# Facebook Data Processing

In [3]:
import pandas as pd
import re
import json
from datetime import datetime

In [4]:
raw_df = pd.read_csv('Raw Data/GOAL.csv')

### Remove columns that do not contain values.

In [5]:
raw_df = raw_df.dropna(axis=1, how='all')

### Remove columns where all the values are identical

In [6]:
raw_df = raw_df.drop_duplicates(subset='post_id')

### Create the first data table.

The data includes the following columns:

- post text
- date
- day
- posting time
- reactions count
- like
- love
- haha
- wow
- sad
- care
- comments
- share

In [7]:
raw_df['new_reactions'] = raw_df['reactions'].apply(lambda x: dict(eval(x)))
s = raw_df['new_reactions'].value_counts()
s

new_reactions
{'like': 4750, 'love': 788, 'haha': 15, 'wow': 2, 'care': 32, 'sad': 1, 'angry': 1}              2
{'like': 1161, 'love': 88, 'wow': 7, 'care': 7, 'sad': 2, 'angry': 1}                            2
{'like': 1295, 'love': 150, 'haha': 5, 'care': 8, 'sad': 1, 'angry': 1}                          2
{'like': 1208, 'love': 16, 'haha': 109, 'wow': 3, 'care': 5, 'sad': 3, 'angry': 1}               2
{'like': 914, 'love': 13, 'haha': 96, 'wow': 5, 'sad': 8, 'angry': 2}                            2
                                                                                                ..
{'like': 60086, 'love': 10592, 'haha': 10296, 'wow': 141, 'care': 994, 'sad': 9, 'angry': 10}    1
{'like': 3753, 'love': 61, 'haha': 677, 'wow': 22, 'care': 14, 'sad': 6, 'angry': 4}             1
{'like': 4003, 'love': 252, 'haha': 9, 'wow': 2, 'care': 13, 'sad': 1, 'angry': 2}               1
{'like': 30914, 'love': 9466, 'haha': 2214, 'wow': 77, 'care': 932, 'sad': 4, 'angry': 3}      

In [8]:
table1_columns = ['text', 'comments', 'shares', 'reaction_count']

table1_df = raw_df[table1_columns].copy()

# Tạo các cột mới từ reactions
table1_df['like'] = raw_df['new_reactions'].apply(lambda x: x.get('like', 0))
table1_df['love'] = raw_df['new_reactions'].apply(lambda x: x.get('love', 0))  
table1_df['haha'] = raw_df['new_reactions'].apply(lambda x: x.get('haha', 0))
table1_df['wow'] = raw_df['new_reactions'].apply(lambda x: x.get('wow', 0))
table1_df['sad'] = raw_df['new_reactions'].apply(lambda x: x.get('sad', 0))  
table1_df['angry'] = raw_df['new_reactions'].apply(lambda x: x.get('angry', 0))

#Tạo cột time
raw_df['time'] = pd.to_datetime(raw_df['time'])

#Chia cột time
table1_df['date'] = raw_df['time'].dt.date
table1_df['day'] = raw_df['time'].dt.day_name()
table1_df['posting_time'] = raw_df['time'].dt.hour

# Hiển thị DataFrame mới
table1_df = pd.DataFrame(table1_df)
table1_df

,text,comments,shares,reaction_count,like,love,haha,wow,sad,angry,date,day,posting_time
0,Which two footballers will be the next 'Messi ...,426.0,19,2078,1507,510,8,1,5,0,2023-11-24,Friday,2
1,Mauricio Sarri gives a hilarious response to t...,6.0,7,585,516,8,59,1,1,0,2023-11-23,Thursday,17
2,Bernardo Silva reveals why he didn't go to Sau...,17.0,5,937,906,26,1,0,1,0,2023-11-23,Thursday,22
3,It's all coming to an end 😢\n\nGOAL.COM\nWhat ...,93.0,27,3810,2963,267,13,7,517,0,2023-11-23,Thursday,20
4,Taiwo Awoniyi will miss AFCON after picking up...,137.0,23,4918,4008,36,19,25,794,8,2023-11-23,Thursday,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,Victor Osimhen achieved what Samuel Eto'o and ...,375.0,79,10000,9182,762,6,10,1,0,2023-09-22,Friday,6
740,How it started. How it ended 3️⃣,487.0,525,10000,7480,1578,48,9,767,0,2023-09-22,Friday,5
741,The Benzema-Vini linkup will never be forgotte...,53.0,24,10000,8835,1047,9,5,59,1,2023-09-22,Friday,4
742,Would he fit in at the Bernabeu? 👀\n\nGOAL.COM...,72.0,8,2439,2209,51,159,7,5,5,2023-09-22,Friday,4


In [9]:
path = "Data/table1.csv"
table1_df.to_csv(path, index=False)
print(path)

Data/table1.csv


### Create the second data table.

In [10]:
import re
from datetime import datetime
import pandas as pd

comments = []
date = []
day = []
post_time = []

for i in range(len(raw_df)):
    s = raw_df.iloc[i]['comments_full'].split("{")
    for c in s:
        if 'comment_text' in c:
            comment = re.search(r"'comment_text': \"(.+?)\", 'comment_time'", c)
            try:
                if comment:
                    comments.append(comment.group(1))
                else:
                    comment = re.search(r"'comment_text': \'(.+?)\', 'comment_time'", c)
                    comments.append(comment.group(1))
                
                t = re.search(r"'comment_time': (.+?)'comment_image", c).group(1)
                dtime = datetime.strptime(t, "datetime.datetime(%Y, %m, %d, %H, %M), ")
                dtime = pd.to_datetime(dtime)
                date.append(dtime.date())
                day.append(dtime.day_name())
                post_time.append(dtime.time())
            except AttributeError as e:
                print(f"Error processing comment: {e}")

Error processing comment: 'NoneType' object has no attribute 'group'
Error processing comment: 'NoneType' object has no attribute 'group'
Error processing comment: 'NoneType' object has no attribute 'group'


In [11]:
table2 = pd.DataFrame({'comment':comments,'date': date,'day': day,'time': post_time})
table2

,comment,date,day,time
0,U see why fans clearly Love Messi especially t...,2023-11-23,Thursday,19:00:00
1,Tyrus Sun lol isn't Messi who misbehave? Chock...,2023-11-23,Thursday,20:00:00
2,Ashif Rahman tor ki mone hoy :(,2023-11-23,Thursday,20:00:00
3,"Tyrus Sun exactly bro,,I fully support you in ...",2023-11-23,Thursday,20:00:00
4,Tyrus Sun why didn't your f00lish father use a...,2023-11-23,Thursday,20:00:00
...,...,...,...,...
31223,Haaland or Mbappe please...not Kane,2023-05-28,Sunday,00:00:00
31224,yes sir,2023-05-28,Sunday,00:00:00
31225,the merchants of success...,2023-05-28,Sunday,00:00:00
31226,Incase you missed this Post ���...just a remin...,2023-05-28,Sunday,00:00:00


In [12]:
path2 = 'Data/table2.csv'
table2.to_csv(path2, index=False)
print(path2)

Data/table2.csv
